In [1]:
import pandas as pd
import numpy as np
import numpy as np
from sklearn import datasets
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from IPython.display import display

In [2]:
# Let us begin by importing two main datasets. 

labelled_data = pd.read_excel('/Users/yahyafarooqi/Documents/AntiGEM/Data/raw/mmc1_Table.S1.xls')

# make small_labels just the Experiment name and a priori MOA categorization 

small_labels = labelled_data[['Experiment name', 'a priori MOA categorization']]

# now delete the everything between - and . in the Experiment name column, so coh2.chip26-batch21.051001 becomes coh2.chip26-051001

display(small_labels)




,Experiment name,a priori MOA categorization
0,coh2.chip26-batch21.051001,RNA
1,col1.chip27-batch21.051001,RNA
2,ril1.chip02-batch24.271101,RNA
3,ril1.chip18-batch26.141201,RNA
4,ril1.chip19-batch27.141201,RNA
...,...,...
306,TuH1.chipXV-batch36.020502,-
307,TuH2.chipXVII-batch36.020502,-
308,TuH2.chipXVIII-batch36.020502,-
309,NaN,NaN


In [3]:
# Load abundance dataset 

abundance_data = pd.read_csv('/Users/yahyafarooqi/Documents/AntiGEM/Data/raw/mmc1_Table.S3.txt', sep='\t')

# Drop the second column
abundance_data = abundance_data.drop(abundance_data.columns[1], axis=1)

abundance_data = abundance_data.T

#make the first row the header
abundance_data.columns = abundance_data.iloc[0]
# delete the first row
abundance_data = abundance_data[1:]

display(abundance_data.head())

Name,ybeF,glpQ,ybdT,ybxH,ybdM,ybdK,ybxF,rplJ,rplK,rpmG,...,yheL,yesN,yclF,dnaA,amhX,ycbT,feuC,gidA,yxnB,spsF
SvL1chip71-batch41.280602,0.7445,0.8471,2.370,0.6920,1.050,1.010,1.220,0.6745,1.230,0.9807,...,-,-,-,-,-,-,-,-,-,-
PheH1.chip37-batch47.181002,-,1.350,5.920,1.220,1.200,0.9780,0.2251,0.1631,0.2823,1.410,...,-,-,-,-,-,-,-,-,-,-
ah1.chip5.200301,0.9311,0.8923,0.9324,0.8720,0.8892,0.7235,0.6688,0.7424,0.9652,0.8565,...,-,-,-,-,-,-,-,-,-,-
ah1.chip7.200301,0.9312,0.8186,0.9549,0.8629,0.8653,0.7351,0.5660,0.6901,0.9101,0.8468,...,-,-,-,-,-,-,-,-,-,-
ah1.chip10.200301,0.8119,0.8766,0.9361,0.8394,0.8769,0.7151,0.5233,0.5883,0.8189,0.9027,...,-,-,-,-,-,-,-,-,-,-


In [4]:
# join by Experiment name and Name 

joined_data = pd.merge(abundance_data, small_labels, left_index=True, right_on='Experiment name')
print(joined_data.shape)
display(joined_data.head())

# remove a priori MOA categorization rows with "-" in them

joined_data = joined_data[joined_data['a priori MOA categorization'] != '-']

print(joined_data.shape)

# save to excel

#joined_data.to_excel('/Users/yahyafarooqi/Documents/AntiGEM/Data/intermediate/abundance_data.xlsx')

(309, 8152)


,ybeF,glpQ,ybdT,ybxH,ybdM,ybdK,ybxF,rplJ,rplK,rpmG,...,yclF,dnaA,amhX,ycbT,feuC,gidA,yxnB,spsF,Experiment name,a priori MOA categorization
8,0.7445,0.8471,2.370,0.6920,1.050,1.010,1.220,0.6745,1.230,0.9807,...,-,-,-,-,-,-,-,-,SvL1chip71-batch41.280602,RNA
241,-,1.350,5.920,1.220,1.200,0.9780,0.2251,0.1631,0.2823,1.410,...,-,-,-,-,-,-,-,-,PheH1.chip37-batch47.181002,AATS
10,0.9311,0.8923,0.9324,0.8720,0.8892,0.7235,0.6688,0.7424,0.9652,0.8565,...,-,-,-,-,-,-,-,-,ah1.chip5.200301,PROTEIN
11,0.9312,0.8186,0.9549,0.8629,0.8653,0.7351,0.5660,0.6901,0.9101,0.8468,...,-,-,-,-,-,-,-,-,ah1.chip7.200301,PROTEIN
9,0.8119,0.8766,0.9361,0.8394,0.8769,0.7151,0.5233,0.5883,0.8189,0.9027,...,-,-,-,-,-,-,-,-,ah1.chip10.200301,PROTEIN


(254, 8152)


In [8]:
# convert all "-" values to 0

joined_data = joined_data.replace('-', 0)

#All values under columns need to be numeric for the model to work, excpet the last two columns
joined_data = joined_data.apply(pd.to_numeric, errors='ignore')

#joined_data.to_excel('/Users/yahyafarooqi/Documents/AntiGEM/Data/intermediate/abundance_data.xlsx')

display(joined_data.head())


/var/folders/hz/wkcmpvw53bb87st6wdf8j_cw0000gp/T/ipykernel_93026/766250041.py:6: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  joined_data = joined_data.apply(pd.to_numeric, errors='ignore')


,ybeF,glpQ,ybdT,ybxH,ybdM,ybdK,ybxF,rplJ,rplK,rpmG,...,yclF,dnaA,amhX,ycbT,feuC,gidA,yxnB,spsF,Experiment name,a priori MOA categorization
8,0.7445,0.8471,2.3700,0.6920,1.0500,1.0100,1.2200,0.6745,1.2300,0.9807,...,0,0,0,0,0,0,0,0,SvL1chip71-batch41.280602,RNA
241,0.0000,1.3500,5.9200,1.2200,1.2000,0.9780,0.2251,0.1631,0.2823,1.4100,...,0,0,0,0,0,0,0,0,PheH1.chip37-batch47.181002,AATS
10,0.9311,0.8923,0.9324,0.8720,0.8892,0.7235,0.6688,0.7424,0.9652,0.8565,...,0,0,0,0,0,0,0,0,ah1.chip5.200301,PROTEIN
11,0.9312,0.8186,0.9549,0.8629,0.8653,0.7351,0.5660,0.6901,0.9101,0.8468,...,0,0,0,0,0,0,0,0,ah1.chip7.200301,PROTEIN
9,0.8119,0.8766,0.9361,0.8394,0.8769,0.7151,0.5233,0.5883,0.8189,0.9027,...,0,0,0,0,0,0,0,0,ah1.chip10.200301,PROTEIN


In [15]:


# remove first column and last two columns in new df called "gene_set"

gene_set = joined_data.iloc[:,1:-2]

# now take average of each column, then make the columns the rows and the rows the columns

gene_set = gene_set.mean(axis=0)

# name first column geneid and second column abundance

gene_set = gene_set.reset_index()

gene_set.columns = ['GeneID', 'Abundance']

# get rid of first column, make first column the index

gene_set = gene_set.iloc[1:]    

gene_set = gene_set.set_index('GeneID')






display(gene_set)

#save to excel


gene_set.to_excel('/Users/yahyafarooqi/Documents/AntiGEM/Data/intermediate/gene_set.xlsx')



,Abundance
GeneID,
ybdT,0.994016
ybxH,0.856044
ybdM,0.963332
ybdK,1.004663
ybxF,1.000806
...,...
ycbT,0.000000
feuC,0.000000
gidA,0.000000


In [60]:
# Let us do SVM on this data, to predict the MOA category

# first we need to split the data into training and testing sets

X = joined_data.drop(['Experiment name', 'a priori MOA categorization'], axis=1)

y = joined_data['a priori MOA categorization']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# now we need to scale the data

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)

X_test = scaler.transform(X_test)

# now we need to train the model

model = SVC()

model.fit(X_train, y_train)

# now we need to predict the test set

y_pred = model.predict(X_test)

# now we need to calculate the accuracy

accuracy = accuracy_score(y_test, y_pred)

# also calculate the confusion matrix. 



print(accuracy)



0.803921568627451
